In [ ]:
# Cell 1 (REVISED): Uninstall and Reinstall Key Libraries

# Step 1: Clean up any potentially conflicting versions
print("--> Uninstalling existing versions...")
!pip uninstall -y transformers accelerate datasets

# Step 2: Reinstall a stable, compatible set of the core libraries
print("\n--> Reinstalling core libraries...")
!pip install transformers accelerate datasets

# Step 3: Install the remaining libraries
print("\n--> Installing other required libraries...")
!pip install scikit-learn pandas imbalanced-learn -q

print("\n✅ All libraries have been reinstalled.")

--> Uninstalling existing versions...
Found existing installation: transformers 4.55.1
Uninstalling transformers-4.55.1:
  Successfully uninstalled transformers-4.55.1
Found existing installation: accelerate 1.10.0
Uninstalling accelerate-1.10.0:
  Successfully uninstalled accelerate-1.10.0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0

--> Reinstalling core libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.7 MB/s eta 0:00:

In [ ]:
# <-- NEW: Check library versions
# This output should be noted for the 'reproducibility' section of the paper
!pip freeze | grep -E "transformers|torch|scikit-learn|imbalanced-learn|datasets"

datasets==4.0.0
imbalanced-learn==0.13.0
scikit-learn==1.6.1
sentence-transformers==5.1.0
tensorflow-datasets==4.9.9
torch @ https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchao==0.10.0
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision @ https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-linux_x86_64.whl
transformers==4.55.2
vega-datasets==0.9.0


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import pickle
import os
import json
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Mount Google Drive
drive.mount('/content/drive')

# --- KEY SETTINGS ---
SEED = 42
GDRIVE_PATH = '/content/drive/MyDrive/eecsi_revise/'
MODEL_NAME = "indobenchmark/indobert-base-p1"

# Set seed for reproducibility across all libraries
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

print(f"✅ Setup complete. Working inside folder: {GDRIVE_PATH}")

Mounted at /content/drive
✅ Setup complete. Working inside folder: /content/drive/MyDrive/eecsi_revise/


In [ ]:
# Define file paths
file_path_csv = os.path.join(GDRIVE_PATH, 'final_golden_dataset_eecsi.csv')
file_path_split = os.path.join(GDRIVE_PATH, 'kfold_splits.pkl')

# Load the dataset and the split file
try:
    df = pd.read_csv(file_path_csv)
    with open(file_path_split, 'rb') as f:
        kfold_splits = pickle.load(f)
    print("✅ Successfully loaded dataset and 5-fold splits.")
except FileNotFoundError as e:
    print(f"❌ ERROR: File not found. Please ensure '{e.filename}' is in the 'eecsi_revise' folder in your Google Drive.")

✅ Successfully loaded dataset and 5-fold splits.


In [ ]:
# Cell 5: Prepare Helper Functions & Classes

# 1. Custom PyTorch Dataset Class
class AspectDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 2. Function to compute metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    report = classification_report(p.label_ids, preds, output_dict=True, zero_division=0)
    # The labels parameter is needed for a consistent report structure
    class_labels = [id2label[i] for i in range(len(id2label))]
    detailed_report = classification_report(p.label_ids, preds, labels=list(range(len(class_labels))), target_names=class_labels, output_dict=True, zero_division=0)
    return {"macro_f1": detailed_report["macro avg"]["f1-score"], "detailed_report": detailed_report}

# 3. Custom Trainer for Weighted Loss
class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    # --- THE CHANGE IS HERE: Added **kwargs ---
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Move class_weights to the same device as logits
        weights = self.class_weights.to(logits.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

print("✅ Helper functions and classes are ready.")

✅ Helper functions and classes are ready.


In [ ]:
# List to store the evaluation results from each fold
fold_results = []
# Create label mappings for consistency
labels = np.array(sorted(df['aspect'].unique()))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

for i, fold in enumerate(kfold_splits):
    print(f"--- Running Fold {i+1}/5 ---")

    # 1. Split data for the current fold
    train_index, test_index = fold['train'], fold['test']
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]

    # 2. Apply Random Oversampling ONLY on the training data
    ros = RandomOverSampler(random_state=SEED)
    X_train_resampled, y_train_resampled = ros.fit_resample(train_df[['cleaned_text']], train_df['aspect'])
    train_df_resampled = pd.concat([X_train_resampled, y_train_resampled], axis=1)

    # 3. Compute Class Weights from the ORIGINAL imbalanced training data for this fold
    class_weights = compute_class_weight('balanced', classes=labels, y=train_df['aspect'])
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

    # 4. Tokenize data
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    train_encodings = tokenizer(list(train_df_resampled['cleaned_text']), truncation=True, padding=True, max_length=128)
    test_encodings = tokenizer(list(test_df['cleaned_text']), truncation=True, padding=True, max_length=128)

    # Convert labels to integers
    train_labels = [label2id[label] for label in train_df_resampled['aspect']]
    test_labels = [label2id[label] for label in test_df['aspect']]

    # Create PyTorch Datasets
    train_dataset = AspectDataset(train_encodings, train_labels)
    test_dataset = AspectDataset(test_encodings, test_labels)

    # 5. Initialize the model for each fold to prevent weight leakage
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(labels),
        label2id=label2id,
        id2label=id2label
    )

    # <-- NEW: Print model parameter count (only for the first fold)
    if i == 0:
        print(f"Model Parameters: {model.num_parameters()/1e6:.2f}M")

    # 6. Define Training Arguments
    training_args = TrainingArguments(
        output_dir=os.path.join(GDRIVE_PATH, f'results_fold_{i+1}'),
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        seed=SEED,
    )

    # 7. Use the custom WeightedLossTrainer
    trainer = WeightedLossTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        class_weights=class_weights_tensor,
    )

    # 8. Train the model
    trainer.train()

    # 9. Evaluate and store the results
    eval_results = trainer.evaluate()
    fold_results.append(eval_results)
    print(f"Fold {i+1} complete. Evaluation results: {eval_results}")

print("\n✅ 5-fold cross-validation process for IndoBERT finished.")

--- Running Fold 1/5 ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model Parameters: 124.45M


Epoch,Training Loss,Validation Loss,Macro F1,Detailed Report
1,0.241600,0.898412,0.702406,"{np.str_('Irrelevant'): {'precision': 0.8768115942028986, 'recall': 0.6080402010050251, 'f1-score': 0.7181008902077152, 'support': 199.0}, np.str_('Smart Economy'): {'precision': 0.5656565656565656, 'recall': 0.9180327868852459, 'f1-score': 0.7, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.8, 'recall': 0.8301886792452831, 'f1-score': 0.8148148148148148, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7246376811594203, 'recall': 0.6944444444444444, 'f1-score': 0.7092198581560284, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.4823529411764706, 'recall': 0.7068965517241379, 'f1-score': 0.5734265734265734, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8421052631578947, 'recall': 0.9230769230769231, 'f1-score': 0.8807339449541285, 'support': 52.0}, np.str_('Smart People'): {'precision': 0.5588235294117647, 'recall': 0.48717948717948717, 'f1-score': 0.5205479452054794, 'support': 39.0}, 'accuracy': 0.7079207920792079, 'macro avg': {'precision': 0.6929125106807162, 'recall': 0.738265581937221, 'f1-score': 0.70240628953782, 'support': 606.0}, 'weighted avg': {'precision': 0.741416577158232, 'recall': 0.7079207920792079, 'f1-score': 0.710021779159007, 'support': 606.0}}"
2,0.052600,1.062671,0.733793,"{np.str_('Irrelevant'): {'precision': 0.8535031847133758, 'recall': 0.6733668341708543, 'f1-score': 0.7528089887640449, 'support': 199.0}, np.str_('Smart Economy'): {'precision': 0.6956521739130435, 'recall': 0.7868852459016393, 'f1-score': 0.7384615384615385, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.8269230769230769, 'recall': 0.8113207547169812, 'f1-score': 0.819047619047619, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.65, 'recall': 0.8125, 'f1-score': 0.7222222222222222, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.6727272727272727, 'recall': 0.6379310344827587, 'f1-score': 0.6548672566371682, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8103448275862069, 'recall': 0.9038461538461539, 'f1-score': 0.8545454545454545, 'support': 52.0}, np.str_('Smart People'): {'precision': 0.6285714285714286, 'recall': 0.5641025641025641, 'f1-score': 0.5945945945945946, 'support': 39.0}, 'accuracy': 0.7392739273927392, 'macro avg': {'precision': 0.7339602806334862, 'recall': 0.7414217981744217, 'f1-score': 0.7337925248960916, 'support': 606.0}, 'weighted avg': {'precision': 0.7514508878061564, 'recall': 0.7392739273927392, 'f1-score': 0.7390635645713386, 'support': 606.0}}"
3,0.004000,1.106318,0.734518,"{np.str_('Irrelevant'): {'precision': 0.7783018867924528, 'recall': 0.8291457286432161, 'f1-score': 0.8029197080291971, 'support': 199.0}, np.str_('Smart Economy'): {'precision': 0.7142857142857143, 'recall': 0.7377049180327869, 'f1-score': 0.7258064516129032, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.8148148148148148, 'recall': 0.8301886792452831, 'f1-score': 0.822429906542056, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7222222222222222, 'recall': 0.7222222222222222, 'f1-score': 0.7222222222222222, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.6976744186046512, 'recall': 0.5172413793103449, 'f1-score': 0.594059405940594, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8867924528301887, 'recall': 0.9038461538461539, 'f1-score': 0.8952380952380953, 'support': 52.0}, np.str_('Smart People'): {'precision': 0.5945945945945946, 'recall': 0.5641025641025641, 'f1-score': 0.5789473684210527, 'support': 39.0}, 'accuracy': 0.7541254125412541, 'macro avg': {'precision': 0.7440980148778056, 'recall': 0.729207377914653, 'f1-score': 0.7345175940008745, 'support': 606.0}, 'weighted avg': {'precision': 0.7514953832404959, 'recall': 0.7541254125412541, 'f1-score': 0.7512058982809946, 'support': 606.0}}"
4,0.017100,1.271081,0.741087,"{np.str_('Irrelevant'): {'precision': 0.845714285

Trainer is attempting to log a value of "{np.str_('Irrelevant'): {'precision': 0.8768115942028986, 'recall': 0.6080402010050251, 'f1-score': 0.7181008902077152, 'support': 199.0}, np.str_('Smart Economy'): {'precision': 0.5656565656565656, 'recall': 0.9180327868852459, 'f1-score': 0.7, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.8, 'recall': 0.8301886792452831, 'f1-score': 0.8148148148148148, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7246376811594203, 'recall': 0.6944444444444444, 'f1-score': 0.7092198581560284, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.4823529411764706, 'recall': 0.7068965517241379, 'f1-score': 0.5734265734265734, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8421052631578947, 'recall': 0.9230769230769231, 'f1-score': 0.8807339449541285, 'support': 52.0}, np.str_('Smart People'): {'precision': 0.5588235294117647, 'recall': 0.48717948717948717, 'f1-score': 0.5205479452054794, 'support': 39.0}, 'a

...

Trainer is attempting to log a value of "{np.str_('Irrelevant'): {'precision': 0.7953216374269005, 'recall': 0.6868686868686869, 'f1-score': 0.7371273712737128, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.6363636363636364, 'recall': 0.8032786885245902, 'f1-score': 0.7101449275362319, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7796610169491526, 'recall': 0.8679245283018868, 'f1-score': 0.8214285714285714, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.83, 'recall': 0.5763888888888888, 'f1-score': 0.680327868852459, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.4927536231884058, 'recall': 0.5862068965517241, 'f1-score': 0.5354330708661418, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8103448275862069, 'recall': 0.8867924528301887, 'f1-score': 0.8468468468468469, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.4583333333333333, 'recall': 0.8461538461538461, 'f1-score': 0.5945945945945946, 'suppo

Epoch,Training Loss,Validation Loss,Macro F1,Detailed Report
1,0.239400,0.903596,0.703700,"{np.str_('Irrelevant'): {'precision': 0.7953216374269005, 'recall': 0.6868686868686869, 'f1-score': 0.7371273712737128, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.6363636363636364, 'recall': 0.8032786885245902, 'f1-score': 0.7101449275362319, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7796610169491526, 'recall': 0.8679245283018868, 'f1-score': 0.8214285714285714, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.83, 'recall': 0.5763888888888888, 'f1-score': 0.680327868852459, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.4927536231884058, 'recall': 0.5862068965517241, 'f1-score': 0.5354330708661418, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8103448275862069, 'recall': 0.8867924528301887, 'f1-score': 0.8468468468468469, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.4583333333333333, 'recall': 0.8461538461538461, 'f1-score': 0.5945945945945946, 'support': 39.0}, 'accuracy': 0.7062706270627063, 'macro avg': {'precision': 0.6861111535496621, 'recall': 0.7505162840171159, 'f1-score': 0.7037004644855083, 'support': 606.0}, 'weighted avg': {'precision': 0.7368595477458906, 'recall': 0.7062706270627063, 'f1-score': 0.7094062337866038, 'support': 606.0}}"
2,0.126400,1.049845,0.722179,"{np.str_('Irrelevant'): {'precision': 0.8414634146341463, 'recall': 0.696969696969697, 'f1-score': 0.7624309392265194, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.7241379310344828, 'recall': 0.6885245901639344, 'f1-score': 0.7058823529411765, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7619047619047619, 'recall': 0.9056603773584906, 'f1-score': 0.8275862068965517, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.6470588235294118, 'recall': 0.8402777777777778, 'f1-score': 0.7311178247734139, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.6666666666666666, 'recall': 0.5172413793103449, 'f1-score': 0.5825242718446602, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8545454545454545, 'recall': 0.8867924528301887, 'f1-score': 0.8703703703703703, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.6176470588235294, 'recall': 0.5384615384615384, 'f1-score': 0.5753424657534246, 'support': 39.0}, 'accuracy': 0.7376237623762376, 'macro avg': {'precision': 0.7304891587340647, 'recall': 0.7248468304102816, 'f1-score': 0.7221792045437309, 'support': 606.0}, 'weighted avg': {'precision': 0.7465105675108012, 'recall': 0.7376237623762376, 'f1-score': 0.7351775227560312, 'support': 606.0}}"
3,0.014200,1.119253,0.720429,"{np.str_('Irrelevant'): {'precision': 0.7892156862745098, 'recall': 0.8131313131313131, 'f1-score': 0.8009950248756219, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.6875, 'recall': 0.7213114754098361, 'f1-score': 0.704, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7586206896551724, 'recall': 0.8301886792452831, 'f1-score': 0.7927927927927928, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7769230769230769, 'recall': 0.7013888888888888, 'f1-score': 0.7372262773722628, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.5689655172413793, 'recall': 0.5689655172413793, 'f1-score': 0.5689655172413793, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8035714285714286, 'recall': 0.8490566037735849, 'f1-score': 0.8256880733944955, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.6388888888888888, 'recall': 0.5897435897435898, 'f1-score': 0.6133333333333333, 'support': 39.0}, 'accuracy': 0.7442244224422442, 'macro avg': {'precision': 0.7176693267934936, 'recall': 0.7248265810619821, 'f1-score': 0.7204287170014122, 'support': 606.0}, 'weighted avg': {'precision': 0.7438811516368854, 'recall': 0.7442244224422442, 'f1-score': 0.7432361134899427, 'support': 606.0}}"
4,0.004000,1.182754,0.741785,"{np.str_('Irrelevant'): {'

Trainer is attempting to log a value of "{np.str_('Irrelevant'): {'precision': 0.8109452736318408, 'recall': 0.8232323232323232, 'f1-score': 0.8170426065162907, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.7192982456140351, 'recall': 0.6721311475409836, 'f1-score': 0.6949152542372882, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7586206896551724, 'recall': 0.8301886792452831, 'f1-score': 0.7927927927927928, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.782608695652174, 'recall': 0.75, 'f1-score': 0.7659574468085106, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.5862068965517241, 'recall': 0.5862068965517241, 'f1-score': 0.5862068965517241, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8363636363636363, 'recall': 0.8679245283018868, 'f1-score': 0.8518518518518519, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.6410256410256411, 'recall': 0.6410256410256411, 'f1-score': 0.6410256410256411, 'suppo

Trainer is attempting to log a value of "{np.str_('Irrelevant'): {'precision': 0.8279569892473119, 'recall': 0.7777777777777778, 'f1-score': 0.8020833333333334, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.7301587301587301, 'recall': 0.7540983606557377, 'f1-score': 0.7419354838709677, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7619047619047619, 'recall': 0.9056603773584906, 'f1-score': 0.8275862068965517, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7569444444444444, 'recall': 0.7569444444444444, 'f1-score': 0.7569444444444444, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.5932203389830508, 'recall': 0.603448275862069, 'f1-score': 0.5982905982905983, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8214285714285714, 'recall': 0.8679245283018868, 'f1-score': 0.8440366972477065, 'support': 53.0}, np.str_('Smart People'): {'precision': 0.6571428571428571, 'recall': 0.5897435897435898, 'f1-score': 0.6216216216

Fold 5 complete. Evaluation results: {'eval_loss': 1.1827536821365356, 'eval_macro_f1': 0.7417854836721748, 'eval_detailed_report': {np.str_('Irrelevant'): {'precision': 0.8279569892473119, 'recall': 0.7777777777777778, 'f1-score': 0.8020833333333334, 'support': 198.0}, np.str_('Smart Economy'): {'precision': 0.7301587301587301, 'recall': 0.7540983606557377, 'f1-score': 0.7419354838709677, 'support': 61.0}, np.str_('Smart Environment'): {'precision': 0.7619047619047619, 'recall': 0.9056603773584906, 'f1-score': 0.8275862068965517, 'support': 53.0}, np.str_('Smart Governance'): {'precision': 0.7569444444444444, 'recall': 0.7569444444444444, 'f1-score': 0.7569444444444444, 'support': 144.0}, np.str_('Smart Living'): {'precision': 0.5932203389830508, 'recall': 0.603448275862069, 'f1-score': 0.5982905982905983, 'support': 58.0}, np.str_('Smart Mobility'): {'precision': 0.8214285714285714, 'recall': 0.8679245283018868, 'f1-score': 0.8440366972477065, 'support': 53.0}, np.str_('Smart People'

In [ ]:
# Extract the 'eval_macro_f1' score from each fold's result
macro_f1_scores = [result['eval_macro_f1'] for result in fold_results]

# Calculate the mean and standard deviation
mean_macro_f1 = np.mean(macro_f1_scores)
std_macro_f1 = np.std(macro_f1_scores)

print("--- Final Aggregated Results (5-Fold CV) for IndoBERT (Improved) ---")
print(f"Macro F1-Score = {mean_macro_f1:.4f} ± {std_macro_f1:.4f}")

--- Final Aggregated Results (5-Fold CV) for IndoBERT (Improved) ---
Macro F1-Score = 0.7453 ± 0.0049


In [ ]:
# Prepare the results dictionary for saving
final_results = {
    'model': 'IndoBERT (Improved)',
    'mean_macro_f1': mean_macro_f1,
    'std_dev_macro_f1': std_macro_f1,
    'results_per_fold': fold_results
}

# Define the output file path
results_file_path = os.path.join(GDRIVE_PATH, 'results_indobert.json')

# Save to a JSON file
with open(results_file_path, 'w') as f:
    json.dump(final_results, f, indent=4)

print(f"\n✅ Final results for IndoBERT have been saved to: '{results_file_path}'")


✅ Final results for IndoBERT have been saved to: '/content/drive/MyDrive/eecsi_revise/results_indobert.json'
